In [4]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GraphConv, TopKPooling
from torch_geometric.nn import global_max_pool as gmp
from torch_geometric.nn import global_mean_pool as gap
import sys

sys.path.append("../")
import utils

train_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TRAIN", True, n_quantiles=100
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TEST", True, n_quantiles=100
)
test_loader = DataLoader(test_dataset, batch_size=64)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = GraphConv(1, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)

        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.log_softmax(self.lin3(x), dim=-1)

        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


def train(epoch):
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


for epoch in range(1, 201):
    loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Loss: {loss:.5f}, Train Acc: {train_acc:.5f}, "
        f"Test Acc: {test_acc:.5f}"
    )



Epoch: 001, Loss: 1.61522, Train Acc: 0.20000, Test Acc: 0.26969
Epoch: 002, Loss: 1.59979, Train Acc: 0.21000, Test Acc: 0.27134
Epoch: 003, Loss: 1.58407, Train Acc: 0.27000, Test Acc: 0.30845
Epoch: 004, Loss: 1.56930, Train Acc: 0.35000, Test Acc: 0.41278
Epoch: 005, Loss: 1.58072, Train Acc: 0.41000, Test Acc: 0.45691
Epoch: 006, Loss: 1.57049, Train Acc: 0.26000, Test Acc: 0.34680
Epoch: 007, Loss: 1.54216, Train Acc: 0.26000, Test Acc: 0.27216
Epoch: 008, Loss: 1.52092, Train Acc: 0.22000, Test Acc: 0.19794
Epoch: 009, Loss: 1.55032, Train Acc: 0.23000, Test Acc: 0.21691
Epoch: 010, Loss: 1.48786, Train Acc: 0.28000, Test Acc: 0.30062
Epoch: 011, Loss: 1.50580, Train Acc: 0.26000, Test Acc: 0.25031
Epoch: 012, Loss: 1.42729, Train Acc: 0.27000, Test Acc: 0.32000
Epoch: 013, Loss: 1.43645, Train Acc: 0.29000, Test Acc: 0.27505
Epoch: 014, Loss: 1.50821, Train Acc: 0.26000, Test Acc: 0.24660
Epoch: 015, Loss: 1.39396, Train Acc: 0.45000, Test Acc: 0.47670
Epoch: 016, Loss: 1.370